In [0]:
'''
%sql
DROP DATABASE dbmovies;
'''

In [0]:
'''
%sql
CREATE DATABASE dbmovies LOCATION "abfss://curated@moviescurated2022.dfs.core.windows.net/DBX_Hive";
'''

In [0]:

'''
%sql
use dbmovies;
--drop table t_Movies;
create table t_Movies(MovieId string, MovieTitle string, Category string, ReleaseDate string, Source string, CreatedDate string, Active string) using Delta;
'''

In [0]:

#in_datepath = getArgument("datepath")
in_datepath = '2021/12/01'
# Read from Raw Layer

df_OpenGate = spark.read.option("header", "true").csv("/mnt/movies_raw/OpenGate/{}/movies.csv".format(in_datepath))
df_WestHighway = spark.read.format("csv").option("header", "true").load("/mnt/movies_raw/WestHighway/{}/Movies.csv".format(in_datepath))
df_BigBrother = spark.read.json("/mnt/movies_raw/BigBrother/{}/".format(in_datepath))

from pyspark.sql.functions import lit
from datetime import datetime
datepart = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

df_OpenGate = df_OpenGate.select('MovieID','MovieTitle','Category','ReleaseDate').withColumn('Source',lit('OpenGate'))
df_WestHighway = df_WestHighway.select('MovieID','MovieTitle','Category','ReleaseDate').withColumn('Source',lit('WestHighway'))
df_BigBrother = df_BigBrother.select('id','title','genre','availabilityDate').withColumn('Source',lit('BigBrother'))

df_final = df_OpenGate.union(df_WestHighway)
df_final = df_final.union(df_BigBrother)
df_final = df_final.withColumn("InsertOn", lit(datepart))
df_final = df_final.withColumn("Active", lit(1))
df_final.createOrReplaceTempView("TBL_Stg_Movies")

spark.sql("Insert into dbmovies.t_Movies select * from TBL_Stg_Movies")

df_final.write.mode("overwrite").parquet('abfss://curated@moviescurated2022.dfs.core.windows.net/movies/')

In [0]:
%sql
select * from dbmovies.t_Movies order by CreatedDate Desc limit 10
--select count(*) from dbmovies.t_Movies

MovieId,MovieTitle,Category,ReleaseDate,Source,CreatedDate,Active
613795b3-a8f5-485c-b096-000d54065d44,Intrigue Cube,Comedy,5/1/2018,WestHighway,2021/12/01 10:23:50,1
74421674-5e11-457a-af51-0012f93c8556,The Robotic Trek,Family,11/14/2017,WestHighway,2021/12/01 10:23:50,1
432daea2-acf3-4643-b64c-00aa72d1ac01,Magic Intrigue,Romance,1/16/2018,WestHighway,2021/12/01 10:23:50,1
7d4a2c91-b0a3-4a0a-be82-00d8f2fec501,Nautical Generation,Action/Adventure,12/12/2017,WestHighway,2021/12/01 10:23:50,1
50ae0384-b603-4756-a7ae-011306f0b3ea,Real Brick,Family,5/22/2018,WestHighway,2021/12/01 10:23:50,1
7b28a010-4cff-4c4b-802b-014c54d94f4e,Journey of Stream,Action/Adventure,2/27/2018,WestHighway,2021/12/01 10:23:50,1
3f6ae89d-eb0d-4aa7-b760-016a0a9a8ae2,The Thread of Hunger,Science Fiction,10/31/2017,WestHighway,2021/12/01 10:23:50,1
6533c76d-177c-4909-8cb9-016d03745c8f,Warehouse of Brick,Science Fiction,11/21/2017,WestHighway,2021/12/01 10:23:50,1
44d39e16-3a45-4009-af91-016f45e18abc,Brick Cluster,Family,6/13/2017,WestHighway,2021/12/01 10:23:50,1
5c4bcb88-6166-4482-9dc0-01d848fc50dd,Robotic Stream,Comedy,4/17/2018,WestHighway,2021/12/01 10:23:50,1
